In [62]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum as quicksum
import numpy as np

In [23]:
def uFunction(k,i): #Working Day probability
    if k in [6, 7, 13, 14]:
        return 0
    else:
        return flatP(t= k-i)

def failureProd(k,i):
    product = 1
    for l in range(i+1, k):
        product = product*(1-uFunction(l, i))
    return product

def constant(r, i):
    total = 0
    for k in range(r, i+8):
        total += uFunction(k,i)*failureProd(k,i)
    return total
        
    
def flatP(t, p0 = 0.318, changeDay = 3):
    if t <= changeDay:
        return p0
    else:
        return (14-t)*p0/(14-changeDay)
    
def alpha_t(a0, a_change, changeDay, t):
    beta_2 = (a0 - a_change)/changeDay
    if t <= changeDay:
        return a0 - beta_2 * t
    else:
        return a0*(14-t)/(14-changeDay) - beta_2 *changeDay* (14 - t)/(14- changeDay) + (t-changeDay)/(14-changeDay)
    
def product(i,j,x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind', 1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    prod = 1
    for k in range (i+1, j):
        prod = prod*(1-tau[k]*flatP(p0, k-i, changeDay))*(x[k]* (alpha_t(a0, a_change, changeDay, j-i) - 1) + 1)
    return prod

def test_product(i,j,x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind',1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    total = 0
    prod = 1
    for t in range(j, i+8): 
        prod = 1
        for k in range(j, t-1):
            prod = prod * (1-tau[k]*flatP(p0, k-i, changeDay))
        prod = prod*tau[t]*flatP(p0, t, changeDay)
        total = total + prod
    return total

def objective(i, x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind',1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    #print(x[i+1])
    total = (x[i+1]*(alpha_t(a0, a_change, changeDay, 1) - 1) + 1)*tau[i+1]*flatP(p0, 1, changeDay)
    for j in range(i+2, i+8):
        total = total + tau[j]*flatP(p0, j-i, changeDay)*(x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1) + 1)*(product(i,j,x))
    return total

def twoTests(i, x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    total = 0
    for j in range(i+1, i+8):
        total = total + x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*test_product(i,j,x)
    for m in range(i+1, i+8):
        for n in range(m+1, i+8):
            #total = total + u[m,n]*(alpha_t(a0, a_change, changeDay, m-i) - 1)*(alpha_t(a0, a_change, changeDay, n-i) - 1)*(product(i,n,x))
            total = total + u[m,n]*(alpha_t(a0, a_change, changeDay, m-i) - 1)*(alpha_t(a0, a_change, changeDay, n-i) - 1)*test_product(i,j,x)
    return total

In [82]:
# Create a new model
m = gp.Model("MIP-test")
i=1
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(i+15, i+15, vtype=GRB.BINARY)
#Create a list of variable x[0] to x[14]
print(x)
risk = twoTests(i, x)
# Set objective:
m.setObjective(risk, GRB.MINIMIZE)
#m.setObjective(-10*x[1] - 5*x[2] - 3*x[1]*u, GRB.MINIMIZE)
# Add constraints
m.addConstr(x[1]+ x[2] + x[3] + x[4]+ x[5] + x[6] + x[7]+ x[8] + x[9] + x[10]+ x[11] + x[12] + x[13] + x[14]== 1 , "c0")
for k in range(i+1, i+8):
    for y in range(k+1, i+8):
        m.addConstr(x[k]*x[y] == u[k,y])

m.optimize()

mylist = []
for v in x.values():
    #print(v.x)
    mylist.append(v.x)
print(mylist)
print(objective(i,mylist))

{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>, 4: <gurobi.Var *Awaiting Model Update*>, 5: <gurobi.Var *Awaiting Model Update*>, 6: <gurobi.Var *Awaiting Model Update*>, 7: <gurobi.Var *Awaiting Model Update*>, 8: <gurobi.Var *Awaiting Model Update*>, 9: <gurobi.Var *Awaiting Model Update*>, 10: <gurobi.Var *Awaiting Model Update*>, 11: <gurobi.Var *Awaiting Model Update*>, 12: <gurobi.Var *Awaiting Model Update*>, 13: <gurobi.Var *Awaiting Model Update*>, 14: <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1 rows, 271 columns and 14 nonzeros
Model fingerprint: 0x3013a02b
Model has 21 quadratic constraints
Variable types: 0 continuous, 271 integer (271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix

In [83]:
#two tests candidate
solSet = []
for t1 in range (0, 2):
    for t2 in range (0,2):
        for t3 in range (0,2):
            for t4 in range (0,2):
                for t5 in range (0,2):
                    for t6 in range(0,2):
                        for t7 in range(0,2):
                            if t1+t2+t3+t4+t5+t6+t7 == 2:
                                solSet.append([t1,t2,t3,t4,t5,t6,t7])
print(solSet)

[[0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0]]


In [108]:
result = []
for i in range(1, 8):
    row = []
    for sol in solSet:
        initSol = ['nal', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        for k in range (7):
            initSol[i+k+1] = sol[k]
        string = []
        for k in range (1, 15):
            if initSol[k] == 1:
                if k >= 8:
                    string.append(k-7)
                else:
                    string.append(k)
        string.sort()
        val = objective(i, initSol)
        row.append([string, val])
        row.sort()
    valList = []
    print(row)
    for item in row:
        valList.append(item[1])
    result.append(valList)
df = pd.DataFrame(result)
df = df.transpose()
df.to_csv('twotests.csv')

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
[[[1, 2], 0.28064544081684745], [[1, 3], 0.4820454408168474], [[1, 4], 0.6266294408168475], [[1, 5], 0.7301660432168474], [[1, 6], 0.7885230372968474], [[1, 7], 0.7885230372968474], [[2, 3], 0.18016906831018076], [[2, 4], 0.22836373497684742], [[2, 5], 0.25942471569684744], [[2, 6], 0.28064544081684745], [[2, 7], 0.28064544081684745], [[3, 4], 0.4297637349768474], [[3, 5], 0.4608247156968474], [[3, 6], 0.4820454408168474], [[3, 7], 0.4820454408168474], [[4, 5], 0.6054087156968474], [[4, 6], 0.6266294408168475], [[4, 7], 0.6266294408168475], [[5, 6], 0.7301660432168474], [[5, 7], 0.7301660432168474], [[6, 7], 0.7885230372968474]]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
[[[1, 2], 0.7339751559348539], [[1, 3], 0.266119847031174], [[1, 4], 0.46751984703117394], [[1, 5], 0.6121038470311739], [[1, 6], 0.7156404494311739], [[1, 7], 0.7156404494311739], [[2, 3], 0.28445455353485394], [[2, 4], 0.4858545535348539], [[2, 5], 0.6304385535348539], [[2, 6]

In [87]:
MonTue = []
MonWed = []
MonThu = []
MonFri = []
MonSat = []
MonSun = []
TueWed = []
TueThu = []
TueFri = []
TueSat = []
TueSun = []
WedThu = []
WedFri = []
WedSat = []
WedSun = []
ThuFri = []
ThuSat = []
ThuSun = []
FriSat = []
FriSun = []
SatSun = []
m1 = 1
m2 = 2
if (m1 == 1 or m1 == 8):
    if (m2 == 2 or m2 ==)
    print('hi')

hi


In [90]:
#Reformulate TwoTest Optimization Problem
m = gp.Model("MIP-test")
i=1
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(i+15, i+15, vtype=GRB.BINARY)
#Create a list of variable x[0] to x[14]
print(x)
risk = twoTests(i, x)
# Set objective:
m.setObjective(risk, GRB.MINIMIZE)
#m.setObjective(-10*x[1] - 5*x[2] - 3*x[1]*u, GRB.MINIMIZE)
# Add constraints
m.addConstr(x[1]+ x[2] + x[3] + x[4]+ x[5] + x[6] + x[7]+ x[8] + x[9] + x[10]+ x[11] + x[12] + x[13] + x[14]== 1 , "c0")
for k in range(i+1, i+8):
    for y in range(k+1, i+8):
        m.addConstr(x[k]*x[y] == u[k,y])

m.optimize()

mylist = []
for v in x.values():
    #print(v.x)
    mylist.append(v.x)
print(mylist)
print(objective(i,mylist))

'mkmkmk'

In [115]:
m = gp.Model("MIP-test")
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(15, 15, vtype=GRB.BINARY)

TypeError: 'tupledict' object is not callable

In [15]:
#Create 2D decision list
DecisionList_2 = []
for i in range (2, 9):
    for j in range (i+1, i+7):
        DecisionList_2.append((i,j))
for i in range (9, 14):
    for j in range (i+1,15):
        DecisionList_2.append((i,j))
DecisionList_2

[(2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 12),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 13),
 (8, 9),
 (8, 10),
 (8, 11),
 (8, 12),
 (8, 13),
 (8, 14),
 (9, 10),
 (9, 11),
 (9, 12),
 (9, 13),
 (9, 14),
 (10, 11),
 (10, 12),
 (10, 13),
 (10, 14),
 (11, 12),
 (11, 13),
 (11, 14),
 (12, 13),
 (12, 14),
 (13, 14)]

In [12]:
DecisionList_3 = []
DecisionList_4 = []
DecisionList_5 = []
DecisionList_6 = []
DecisionList_7 = []
for i in range (2, 9):
    for j in range (i+1, i+6):
        for k in range (j+1, i+7):
            DecisionList_3.append((i,j,k))

for i in range (9, 13):
    for j in range (i+1,14):
        for k in range (j+1, 15):
            DecisionList_3.append((i,j,k))
            
for l1 in range(2, 9):
    for l2 in range(l1 + 1, l1+5):
        for l3 in range(l2+1, l1+6):
            for l4 in range (l3 + 1, l1+7):
                DecisionList_4.append((l1,l2,l3,l4))
                
for l1 in range(9, 12):
    for l2 in range(l1 + 1, 13):
        for l3 in range(l2+1, 14):
            for l4 in range (l3 + 1, 15):
                DecisionList_4.append((l1,l2,l3,l4))

for l1 in range(2, 9):
    for l2 in range(l1 + 1, l1+4):
        for l3 in range(l2+1, l1+5):
            for l4 in range (l3 + 1, l1+6):
                for l5 in range (l4+1, l1+7):
                    DecisionList_5.append((l1,l2,l3,l4,l5))
                
for l1 in range(9, 11):
    for l2 in range(l1 + 1, 12):
        for l3 in range(l2+1, 13):
            for l4 in range (l3 + 1, 14):
                for l5 in range (l4+1, 15):
                    DecisionList_5.append((l1,l2,l3,l4,l5))

for l1 in range(2, 9):
    for l2 in range(l1 + 1, l1+3):
        for l3 in range(l2+1, l1+4):
            for l4 in range (l3 + 1, l1+5):
                for l5 in range (l4+1, l1+6):
                    for l6 in range (l5+1, l1+7):
                        DecisionList_6.append((l1,l2,l3,l4,l5, l6))
                
for l1 in range(9, 10):
    for l2 in range(l1 + 1, 11):
        for l3 in range(l2+1, 12):
            for l4 in range (l3 + 1, 13):
                for l5 in range (l4+1, 14):
                    for l6 in range (l5+1, 15):
                        DecisionList_6.append((l1,l2,l3,l4,l5,l6))

for l1 in range(2, 9):
    for l2 in range(l1 + 1, l1+2):
        for l3 in range(l2+1, l1+3):
            for l4 in range (l3 + 1, l1+4):
                for l5 in range (l4+1, l1+5):
                    for l6 in range (l5+1, l1+6):
                        for l7 in range (l6+1, l1+7):
                            DecisionList_7.append((l1,l2,l3,l4,l5, l6, l7))
                
for l1 in range(10, 10):
    for l2 in range(l1 + 1, 10):
        for l3 in range(l2+1, 11):
            for l4 in range (l3 + 1, 12):
                for l5 in range (l4+1, 13):
                    for l6 in range (l5+1, 14):
                        for l7 in range (l6+1, 15):
                            DecisionList_7.append((l1,l2,l3,l4,l5,l6,l7))

In [13]:
DecisionList_7

[(2, 3, 4, 5, 6, 7, 8),
 (3, 4, 5, 6, 7, 8, 9),
 (4, 5, 6, 7, 8, 9, 10),
 (5, 6, 7, 8, 9, 10, 11),
 (6, 7, 8, 9, 10, 11, 12),
 (7, 8, 9, 10, 11, 12, 13),
 (8, 9, 10, 11, 12, 13, 14)]

In [71]:
#Initialize
changeDay = 3
a0=0.4
a_change=0.3
p0=0.318

#Gurobi optimization 
m = gp.Model("MIP-test")
x = m.addVars(15, vtype=GRB.BINARY)
y2 = m.addVars(15, 15, vtype=GRB.BINARY)
y3 = m.addVars(15, 15, 15, vtype=GRB.BINARY)
z = m.addVar(vtype=GRB.CONTINUOUS, name="z")
#Create a list of variable x[0] to x[14]
#Create a list of variable y[0,0] to y[14,14]
for decision in twoDecisionList:
    j = decision[0]
    r = decision[1]
    m.addConstr(y2[j,r] <= x[j], 'c%s 1'%(str(decision)))
    m.addConstr(y2[j,r] <= x[r], 'c%s 2'%(str(decision)))
    m.addConstr(y2[j,r] >= x[r] + x[j] - 1, 'c%s 3'%(str(decision)))

for i in range(1, 8):
    m.addConstr(z >= x[i+1]*(alpha_t(a0, a_change, changeDay, 1) - 1)*uFunction(i+1, i) + uFunction(i+1, i) + quicksum(x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*constant(j,i) for j in range(i+1, i+8)) + quicksum(y2[j,r]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*(alpha_t(a0, a_change, changeDay, r-i) - 1)*constant(r,i) for j in range(i+1, i+7) for r in range(j+1, i+8)) + constant(i+2, i), "myconstraint")

m.addConstr(quicksum(x[k] for k in range(15)) == 2)
m.setObjective(z, GRB.MINIMIZE)
m.optimize()

NameError: name 'y' is not defined

In [147]:
mylist = []
obj = m.getObjective().getValue()
for v in x.values():
    mylist.append(v.x)
print('Integer Solution:')
print(mylist)
print('Objective Value:')
print(obj)

Integer Solution:
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Objective Value
0.6143520166745567


In [45]:
#Resampling multinomial
def mltSamples(n, i):
    u1 = uFunction(i+1, i)
    u2 = uFunction(i+2, i)
    u3 = uFunction(i+3, i)
    u4 = uFunction(i+4, i)
    u5 = uFunction(i+5, i)
    u6 = uFunction(i+6, i)
    u7 = uFunction(i+7, i)
    pList = [u1, (1-u1)*u2, (1-u1)*(1-u2)*u3, (1-u1)*(1-u2)*(1-u3)*u4, (1-u1)*(1-u2)*(1-u3)*(1-u4)*u5, (1-u1)*(1-u2)*(1-u3)*(1-u4)*(1-u5)*u6, (1-u1)*(1-u2)*(1-u3)*(1-u4)*(1-u5)*(1-u6)*u7, (1-u1)*(1-u2)*(1-u3)*(1-u4)*(1-u5)*(1-u6)*(1-u7)]
    print(pList)
    s = np.random.multinomial(n, pList, size=1000)
    return s


[0.318, 0.21687599999999999, 0.14790943199999998, 0.0, 0.0, 0.07336307827199998, 0.049346674194047986, 0.19450481553395196]


In [50]:
def quick_prod(a0, a_change, changeDay, i, j):
    prod = 1
    for k in range(i+1, j+1):
        prod = prod * (alpha_t(a0, a_change, changeDay, k) - 1)
    return prod

def two_prod(a0, a_change, changeDay, a, b):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    return prod

def three_prod(a0, a_change, changeDay, a, b, c):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, c) - 1)
    return prod

def four_prod(a0, a_change, changeDay, a, b, c, d):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, c) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, d) - 1)
    return prod

def five_prod(a0, a_change, changeDay, a, b, c, d, e):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, c) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, d) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, e) - 1)
    return prod

def six_prod(a0, a_change, changeDay, a, b, c , d, e, f):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, c) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, d) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, e) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, f) - 1)
    return prod

def sev_prod(a0, a_change, changeDay, a, b, c, d, e, f, g):
    prod = 1
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, a) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, b) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, c) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, d) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, e) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, f) - 1)
    prod = prod * (alpha_t(a0, a_change, changeDay, g) - 1)
    return prod

In [87]:
#Initialize
changeDay = 3
a0=0.4
a_change=0.3
p0=0.318
n=50
beta = 0.99
omega = 5
K = 1000
#Opitmization Setting 
m = gp.Model("MIP-test")
x = m.addVars(15, vtype=GRB.BINARY)
y2 = m.addVars(DecisionList_2, vtype=GRB.BINARY)
y3 = m.addVars(DecisionList_3, vtype=GRB.BINARY)
y4 = m.addVars(DecisionList_4, vtype=GRB.BINARY)
y5 = m.addVars(DecisionList_5, vtype=GRB.BINARY)
y6 = m.addVars(DecisionList_6, vtype=GRB.BINARY)
y7 = m.addVars(DecisionList_7, vtype=GRB.BINARY)
z = m.addVars(8, K, vtype = GRB.CONTINUOUS)
c = m.addVars(8, vtype =GRB.CONTINUOUS)

#Regularity constraints
for decision in DecisionList_2:
    j = decision[0]
    r = decision[1]
    m.addConstr(y2[j,r] <= x[j], 'c%s 1'%(str(decision)))
    m.addConstr(y2[j,r] <= x[r], 'c%s 2'%(str(decision)))
    m.addConstr(y2[j,r] >= x[r] + x[j] - 1, 'c%s 3'%(str(decision)))

for decision in DecisionList_3:
    l1 = decision[0]
    l2 = decision[1]
    l3 = decision[2]
    m.addConstr(y3[l1, l2, l3] <= y2[l1, l2], 'c%s 1'%(str(decision)))
    m.addConstr(y3[l1, l2, l3] <= x[l3], 'c%s 2'%(str(decision)))
    m.addConstr(y3[l1, l2, l3] >= y2[l1, l2] + x[l3] - 1, 'c%s 3'%(str(decision)))

for decision in DecisionList_4:
    l1 = decision[0]
    l2 = decision[1]
    l3 = decision[2]
    l4 = decision[3]
    m.addConstr(y4[l1, l2, l3, l4] <= y3[l1, l2, l3], 'c%s 1'%(str(decision)))
    m.addConstr(y4[l1, l2, l3, l4] <= x[l4], 'c%s 2'%(str(decision)))
    m.addConstr(y4[l1, l2, l3, l4] >= y3[l1, l2, l3] + x[l4] - 1, 'c%s 3'%(str(decision)))

for decision in DecisionList_5:
    l1 = decision[0]
    l2 = decision[1]
    l3 = decision[2]
    l4 = decision[3]
    l5 = decision[4]
    m.addConstr(y5[l1, l2, l3, l4, l5] <= y4[l1, l2, l3, l4], 'c%s 1'%(str(decision)))
    m.addConstr(y5[l1, l2, l3, l4, l5] <= x[l5], 'c%s 2'%(str(decision)))
    m.addConstr(y5[l1, l2, l3, l4, l5] >= y4[l1, l2, l3, l4] + x[l5] - 1, 'c%s 3'%(str(decision)))

for decision in DecisionList_6:
    l1 = decision[0]
    l2 = decision[1]
    l3 = decision[2]
    l4 = decision[3]
    l5 = decision[4]
    l6 = decision[5]
    m.addConstr(y6[l1, l2, l3, l4, l5, l6] <= y5[l1, l2, l3, l4, l5], 'c%s 1'%(str(decision)))
    m.addConstr(y6[l1, l2, l3, l4, l5, l6] <= x[l6], 'c%s 2'%(str(decision)))
    m.addConstr(y6[l1, l2, l3, l4, l5, l6] >= y5[l1, l2, l3, l4, l5] + x[l6] - 1, 'c%s 3'%(str(decision)))

for decision in DecisionList_7:
    l1 = decision[0]
    l2 = decision[1]
    l3 = decision[2]
    l4 = decision[3]
    l5 = decision[4]
    l6 = decision[5]
    l7 = decision[6]
    m.addConstr(y7[l1, l2, l3, l4, l5, l6, l7] <= y6[l1, l2, l3, l4, l5, l6], 'c%s 1'%(str(decision)))
    m.addConstr(y7[l1, l2, l3, l4, l5, l6, l7] <= x[l7], 'c%s 2'%(str(decision)))
    m.addConstr(y7[l1, l2, l3, l4, l5, l6, l7] >= y6[l1, l2, l3, l4, l5, l6] + x[l7] - 1, 'c%s 3'%(str(decision)))

for i in range(1,8):
    m.addConstr(x[i] == x[i+7], 'symmetry%s'%i)
    
#CVaR Constraints
for i in range(1, 8):
    samples = mltSamples(n, i)
    for k in range (K):
        j1 = samples[k][0]
        j2 = samples[k][1]
        j3 = samples[k][2]
        j4 = samples[k][3]
        j5 = samples[k][4]
        j6 = samples[k][5]
        j7 = samples[k][6]
        e1 = j1*(x[i+1]*(alpha_t(a0, a_change, changeDay, i+1)-1) + 1)
        e2 = j2*(y2[i+1, i+2]*(alpha_t(a0, a_change, changeDay, i+1)-1)*(alpha_t(a0, a_change, changeDay, i+2)-1) + x[i+1]*(alpha_t(a0, a_change, changeDay, i+1)-1) + x[i+2]*(alpha_t(a0, a_change, changeDay, i+2)-1) +1)
        e3 = j3*(y3[i+1, i+2, i+3]*quick_prod(a0, a_change, changeDay, i, i+3) + quicksum(y2[m,n]*two_prod(a0, a_change, changeDay, m, n) for m in range(i+1, i+2) for n in range(m+1, i+4)) + quicksum(x[m]*(alpha_t(a0, a_change, changeDay, m) - 1) for m in range(i+1, i+4)) +1)
        e4 = j4*(y4[i+1, i+2, i+3, i+4]*quick_prod(a0, a_change, changeDay, i, i+4)+ quicksum(y3[m,n,o]*three_prod(a0, a_change, changeDay, m, n, o) for m in range(i+1, i+2) for n in range (m+1, i+3) for o in range (n+1, i+5) ) + quicksum(y2[m,n]*two_prod(a0, a_change, changeDay, m, n) for m in range(i+1, i+3) for n in range(m+1, i+5) ) + quicksum(x[m]*(alpha_t(a0, a_change, changeDay, m) - 1) for m in range(i+1, i+5)) +1)
        e5 = j5*(y5[i+1, i+2, i+3, i+4, i+5]*quick_prod(a0, a_change, changeDay, i, i+5)+ quicksum(y4[m,n,o,p]*four_prod(a0, a_change, changeDay, m, n, o, p) for m in range(i+1, i+2) for n in range(m+1, i+3) for o in range(n+1, i+4) for p in range(o+1, i+6)) +quicksum(y3[m,n,o]*three_prod(a0, a_change, changeDay, m, n, o) for m in range(i+1, i+3) for n in range (m+1, i+4) for o in range (n+1, i+6)) + quicksum(y2[m,n]*two_prod(a0, a_change, changeDay, m, n) for m in range(i+1, i+4) for n in range(m+1, i+6)) + quicksum(x[m]*(alpha_t(a0, a_change, changeDay, m) - 1) for m in range(i+1, i+6)) +1)
        e6 = j6*(y6[i+1, i+2, i+3, i+4, i+5, i+6]*quick_prod(a0, a_change, changeDay, i, i+6)+ quicksum(y5[m,n,o,p,q]*five_prod(a0, a_change, changeDay, m, n, o, p, q) for m in range(i+1, i+2) for n in range (m+1, i+3) for o in range(n+1, i+4) for p in range(o+1, i+5) for q in range(p+1, i+7)) +quicksum(y4[m,n,o,p]*four_prod(a0, a_change, changeDay, m, n, o, p) for m in range(i+1, i+3) for n in range(m+1, i+4) for o in range(n+1, i+5) for p in range(o+1, i+7)) +quicksum(y3[m,n,o]*three_prod(a0, a_change, changeDay, m, n, o) for m in range(i+1, i+4) for n in range (m+1, i+5) for o in range (n+1, i+7)) + quicksum(y2[m,n]*two_prod(a0, a_change, changeDay, m, n) for m in range(i+1, i+5) for n in range(m+1, i+7)) + quicksum(x[m]*(alpha_t(a0, a_change, changeDay, m) - 1) for m in range(i+1, i+7)) +1)
        e7 = j7*(y7[i+1, i+2, i+3, i+4, i+5, i+6, i+7]*quick_prod(a0, a_change, changeDay, i, i+6)+ quicksum(y6[m,n,o,p,q,r]*six_prod(a0, a_change, changeDay, m, n, o, p, q, r) for m in range(i+1, i+2) for n in range(m+1, i+3) for o in range (n+1, i+4) for p in range (o+1, i+5) for q in range (p+1, i+6) for r in range (q+1, i+8)) +quicksum(y5[m,n,o,p,q]*five_prod(a0, a_change, changeDay, m, n, o, p, q) for m in range(i+1, i+3) for n in range (m+1, i+4) for o in range(n+1, i+5) for p in range(o+1, i+6) for q in range(p+1, i+8)) +quicksum(y4[m,n,o,p]*four_prod(a0, a_change, changeDay, m, n, o, p) for m in range(i+1, i+4) for n in range(m+1, i+5) for o in range(n+1, i+6) for p in range(o+1, i+8)) +quicksum(y3[m,n,o]*three_prod(a0, a_change, changeDay, m, n, o) for m in range(i+1, i+5) for n in range (m+1, i+6) for o in range (n+1, i+8)) + quicksum(y2[m,n]*two_prod(a0, a_change, changeDay, m, n) for m in range(i+1, i+6) for n in range(m+1, i+8)) + quicksum(x[m]*(alpha_t(a0, a_change, changeDay, m) - 1) for m in range(i+1, i+8)) +1)
        m.addConstr(z[i, k] >= 0, 'ReLU1%s%s'%(i,k))
        m.addConstr(z[i, k] >= e1+e2+e3+e4+e5+e6+e7-c[i], 'ReLU2%s%s'%(i,k))
    m.addConstr(c[i] + (quicksum(z[i,k] for k in range(K)))/((1-beta)*K) <= omega)
m.setObjective(quicksum(x[k] for k in range(15)), GRB.MINIMIZE)
m.optimize()

[0.318, 0.21687599999999999, 0.14790943199999998, 0.09170384783999999, 0.0, 0.0, 0.04563516937055999, 0.17987555078943998]
[0.318, 0.21687599999999999, 0.14790943199999998, 0.0, 0.0, 0.07336307827199998, 0.049346674194047986, 0.19450481553395196]
[0.318, 0.21687599999999999, 0.0, 0.0, 0.12101680799999998, 0.07958260876799998, 0.0535301565704029, 0.21099442666159707]
[0.318, 0.0, 0.0, 0.19716, 0.12614655272727274, 0.08295601180561983, 0.05579923012270738, 0.2199382053444001]
[0, 0, 0.318, 0.19716, 0.12614655272727274, 0.08295601180561983, 0.05579923012270738, 0.2199382053444001]
[0, 0.318, 0.21687599999999999, 0.13446312, 0.08603194895999998, 0.056576000051432714, 0.0, 0.18805293098856724]
[0.318, 0.21687599999999999, 0.14790943199999998, 0.09170384783999999, 0.058673789190719985, 0.0, 0.0, 0.16683693096927996]
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 15508 rows, 8521 colum

In [86]:
mylist = []
obj = m.getObjective().getValue()
for v in x.values():
    mylist.append(v.x)
print('Integer Solution:')
print(mylist)
print('Objective Value:')
print(obj)

Integer Solution:
[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]
Objective Value:
8.0
